<a href="https://colab.research.google.com/github/pengyk/COMP400/blob/master/Comp400.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rasterio

In [ ]:
import json
import random
import numpy as np
import pandas as pd
import copy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
from os import listdir
from os.path import isfile, join
import re
import rasterio
from rasterio import Affine, features
from shapely.geometry import mapping, shape, Polygon, Point
from shapely.ops import cascaded_union
from math import floor, ceil, sqrt
from google.colab import files
from sklearn.cluster import MeanShift, estimate_bandwidth, DBSCAN, KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from itertools import cycle
from sklearn.preprocessing import StandardScaler
import scipy.stats as st
from sklearn import metrics

# global paths
base_path = "drive/My Drive/yunkai/"
data_path = base_path + 'data'
solutions_path = base_path + "solutions.csv"
scaled_solutions_path = base_path + "scaled_solutions.csv"
output_path = base_path + "output"
path_to_consensus_json = base_path + "consensus.json"
clrs_path = base_path + "clrs"
consensus_path = base_path + "consensus"
average_polygon_path = base_path + "average_polygon"

np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)


## Script for scaling up to original size the coordinates for the games played by users

In [ ]:
def scale_solutions():

  df = pd.read_csv(solutions_path, names=['filepath','score','result','minmax','player','created_at'], header=0, sep="','", dtype={"score" :np.float64})
  df["filepath"] = df["filepath"].str[1:]
  df["created_at"] = df["created_at"].str[:-1]
  df["result"] = df["result"].apply(json.loads)
  df["minmax"] = df["minmax"].apply(json.loads)
  df["scaled_polygons"] = ""
  df["mins"] = ""
  df["maxs"] = ""
  
  for index, row in df.iterrows():
    scaled_polygons = []
    polygons = np.array(df['result'][index]['polygons'])
    df['mins'][index] = df['minmax'][index][:2]
    mins = np.array(df['minmax'][index][:2])
    df['maxs'][index] = df['minmax'][index][2:]
    maxs = np.array(df['minmax'][index][2:])
    for polygon in polygons:
      output = np.empty((0,2), type)
      for vertex in polygon['vertices']:
        vertex = mins + (vertex * (maxs-mins))
        if len(output) == 0:
            output = vertex
        else:
          output = np.vstack((output, vertex))
      scaled_polygons.append(output.tolist())
    df["scaled_polygons"][index] = scaled_polygons
  df.to_csv(scaled_solutions_path)


# Consensus polygon generation based on K-Means with Average Silhouette




In [ ]:
def random_distribution_consensus_generator(max_number_clusters = 5, x_shrink = 0.4, y_shrink = 0.4):
  
  # The generation will use all the available datasets that we have.
  # It will traverse all the folders and all the files present in the clrs folder
  # in order to obtain all the datasets.
  folders = listdir(clrs_path)
  for folder in folders:
    all_files = listdir(clrs_path + '/' + folder)
    for file_name in all_files:
      file_name_without_csv = re.sub('\.csv', '', file_name)
      full_file_path = folder + '/' + file_name_without_csv

      # This function fetches all games played on a specified dataset
      polygons = random_plot_get_all_games(full_file_path)

      # obtain the max and min coordinate to scale the value down to a 1x1 square
      minX = 2147483647
      minY = 2147483647
      maxX = -2147483647
      maxY = -2147483647
      for polygon in polygons:
        for coordinates in polygon:
          minX = min(minX, coordinates[0])
          maxX = max(maxX, coordinates[0])
          minY = min(minY, coordinates[1])
          maxY = max(maxY, coordinates[1])

      # scale the coordinates between the range of [0,1]
      scaled_polygons = []
      for polygon in polygons:
        temp_list = []
        for coordinates in polygon:
          x_val = (coordinates[0] - minX)/ (maxX - minX)
          y_val = (coordinates[1] - minY)/ (maxY - minY)
          temp_list.append(tuple([x_val, y_val]))
        scaled_polygons.append(temp_list)

      # If there is less than 30 games played, do not generate the consensus polygon as it is too small sample size
      if len(polygons) < 30 :
        continue

      polygons = scaled_polygons

      # Get random points per polygon in order to later plot density plot
      scatter_plot = []

      # shrink down size of each polygon to get a better separation
      for polygon in polygons:
        xs, ys = zip(*polygon)
        xs = np.array(xs)
        ys = np.array(ys)
        x_center = 0.5 * min(xs)  + 0.5 * max(xs)
        y_center = 0.5 * min(ys)  + 0.5 * max(ys)
        new_xs = [(i - x_center) * (1 - x_shrink) + x_center for i in xs]
        new_ys = [(i - y_center) * (1 - y_shrink) + y_center for i in ys]
        # create list of new coordinates
        polygon = zip(new_xs, new_ys)
        poly = Polygon(polygon)
        polygonArea = PolyArea(xs,ys)
        for i in range(int(100 * (polygonArea))):
          point_in_poly = random_point_in_polygon(poly)
          scatter_plot.append([point_in_poly.x, point_in_poly.y])
        
      # Store it in a numpy array so that it is easier to fetch all x coordinates and all y coordinates later
      X = np.array(scatter_plot)

      # Try to get the highest average silhouette and keep track of the number of clusters that gives it
      # Start iteration with 2 clusters and go up until the max_number_clusters
      max_average = 0
      best_cluster_number = 0
      n_clusters = 2

      while n_clusters <= max_number_clusters:
        # calculate the highest Average Silhouette score
        clusterer = KMeans(n_clusters=n_clusters)
        cluster_labels = clusterer.fit_predict(X)
        silhouette_avg = silhouette_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              "The average silhouette_score is :", silhouette_avg)
        if max_average < silhouette_avg:
          max_average = silhouette_avg
          best_cluster_number = n_clusters
        n_clusters += 1

      # Now that we have the number of clusters that generates the highest
      # average silhouette score, we can plot the graphs.
      n_clusters = best_cluster_number

      # Clear graph just to be sure.
      plt.cla()
      plt.close()
      plt.clf()
      fig, (ax1, ax2) = plt.subplots(1, 2)
      fig.set_size_inches(18, 7)

      # The silhouette plot that ranges from -0.1 to 1.
      ax1.set_xlim([-0.1, 1])
      # Add a little space between average silhouette graphs.
      ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

      # Initialize the clusterer with n_clusters value
      clusterer = KMeans(n_clusters=n_clusters)
      cluster_labels = clusterer.fit_predict(X)
      # The silhouette_score gives the average value for all the samples.
      # This gives a perspective into the density and separation of the formed
      # clusters
      silhouette_avg = silhouette_score(X, cluster_labels)
      print("For n_clusters =", n_clusters,
            "The best average silhouette_score is :", silhouette_avg)
      # Compute the silhouette scores for each sample
      sample_silhouette_values = silhouette_samples(X, cluster_labels)

      y_lower = 10
      for i in range(n_clusters):
        # Separate points based on cluster
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

      ax1.set_title("The silhouette plot for the various clusters.")
      ax1.set_xlabel("The silhouette coefficient values")
      ax1.set_ylabel("Cluster label")

      # The vertical line for average silhouette score of all the values
      ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

      ax1.set_yticks([])  # Clear the labels
      ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

      # Plot showing the clusters.
      colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
      ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,c=colors, edgecolor='k')
      plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                    "with n_clusters = %d" % n_clusters),
                  fontsize=14, fontweight='bold')
      plt.show()

      # Separate the data into different clusters, since there are varying densities,
      # it is more precise if we can computer the contour for each cluster/polygon

      # clusters is the dictionnary that contains the mapping between each cluster and its points
      clusters = dict()

      for i in range(len(colors)):
        # We use the color as the key
        checksum = np.array_str(colors[i])
        if not checksum in clusters:
          clusters[checksum] = []
        clusters[checksum].append(X[i].tolist())

      # exterior bounds will save all the outer contours for all the polygons in the plot
      exterior_bounds = []

      for cluster in clusters:
        plt.cla()
        plt.clf()
        coordinates = np.array(clusters[cluster])
        x = coordinates[:, 0]
        y = coordinates[:, 1]

        # Define the borders
        values = np.vstack([x, y])
        kernel = st.gaussian_kde(values)

        deltaX = (max(x) - min(x))/10
        deltaY = (max(y) - min(y))/10
        xmin = min(x) - deltaX
        xmax = max(x) + deltaX
        ymin = min(y) - deltaY
        ymax = max(y) + deltaY

        # Create meshgrid
        xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j] 

        positions = np.vstack([xx.ravel(), yy.ravel()])
        values = np.vstack([x, y])
        f = np.reshape(kernel(positions).T, xx.shape)
        fig = plt.figure(figsize=(8,8))
        ax = fig.gca()
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        cfset = ax.contourf(xx, yy, f, cmap='coolwarm')
        ax.imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
        cset = ax.contour(xx, yy, f, colors='k')
        ax.clabel(cset, inline=1, fontsize=10)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        plt.title('2D Gaussian Kernel density estimation')
        plt.figure(figsize=(8,8))
        for clust in range(len(cset.allsegs)):
          largest_segment = []
          for lev, seg in enumerate(cset.allsegs[clust]):
            # only the most outer level matters
            if clust == 1:
              # for a cluster, get the largest of the two
              if len(largest_segment) == 0:
                largest_segment = seg
              else:
                largest_xs, largest_ys = zip(*largest_segment)
                largest_xs = np.array(largest_xs)
                largest_ys = np.array(largest_ys)

                seg_xs, seg_ys = zip(*seg)
                seg_xs = np.array(seg_xs)
                seg_ys = np.array(seg_ys)

                if PolyArea(largest_xs, largest_ys) < PolyArea(seg_xs, seg_ys):
                  largest_segment = seg
              # save it to the array
            plt.plot(seg[:,0], seg[:,1], '.-', label=f'Cluster{clust}, level{lev}')
          if clust == 1:
            exterior_bounds.append(largest_segment)
        plt.legend()
        plt.show()
      # rescale back to original size
      original_sized_bound = []
      for i in range(len(exterior_bounds)):
        original_sized_bound.append([])
        polygon = exterior_bounds[i]
        for j in range(len(polygon)):
          coordinates = polygon[j]
          original_sized_bound[i].append([])
          original_sized_bound[i][j].append(coordinates[0] * (maxX - minX) + minX)
          original_sized_bound[i][j].append(coordinates[1] * (maxY - minY) + minY)
      
      np.save(average_polygon_path + '/' + folder + '~' + file_name_without_csv + '~consensus', original_sized_bound)

# Polygon area calculator

In [ ]:
def PolyArea(x,y):
  return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

# Calculate max area of all the polygons for a file

In [ ]:
def getTotalArea(polygons):
  minX = 2147483647
  minY = 2147483647
  maxX = -2147483647
  maxY = -2147483647
  for polygon in polygons:
    for coordinates in polygon:
      minX = min(minX, coordinates[0])
      maxX = max(maxX, coordinates[0])
      minY = min(minY, coordinates[1])
      maxY = max(maxY, coordinates[1])
  return (maxX - minX) * (maxY - minY)

# Function to randomly fill each polygon with points

In [ ]:
def random_point_in_polygon(poly):
  minx, miny, maxx, maxy = poly.bounds
  while True:
    p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
    if poly.contains(p):
      return p 

# Get all games played for each graph/plot

In [ ]:
def random_plot_get_all_games(full_file_path):
  print(full_file_path)
  full_file_path_csv = full_file_path + '.csv'
  path = "drive/My Drive/yunkai/solutions.csv"

  df = pd.read_csv(scaled_solutions_path)
  
  # get all games of this dataset from all players
  output = []
  number_of_games = 0;
  for index, row in df.iterrows():
    if row['filepath'] == full_file_path_csv:
      number_of_games += 1
      read_json  = json.loads(row['scaled_polygons'])
      for coord in read_json:
        output.append(coord)
  return output

# Test to visualize consensus polygons

In [ ]:
def double_check_consensus():
  # np.load.__defaults__=(None, True, True, 'ASCII')
  all_files = listdir(average_polygon_path)
  for file_name in all_files:
    print(average_polygon_path + '/' + file_name)
    X = np.load(average_polygon_path + '/' + file_name)
    plt.clf()
    plt.cla()
    for x in X:
      x = np.array(x)
      plt.plot(x[:,0], x[:,1])
    plt.title("Consensus generation of " + str(file_name))
    plt.show()
  

# Main

In [ ]:
scale_solutions()
random_distribution_consensus_generator()
double_check_consensus()

# Archived code

### Check if it necessary to remove one layer of parentheses

In [ ]:
def remove_layer_of_parenthesis(list_of_tuples):
  if len(list(list(list_of_tuples)[0])) == 2:
    return False
  else :
    return True

## Code for plotting

In [ ]:
def plotGraph(folder, fileName, outputPath):
  pathToData = "drive/My Drive/yunkai/data/" + folder + '/' + fileName + ".npy"
  pathToCluster = "drive/My Drive/yunkai/clrs/" + folder + '/' + fileName
  pathToSolutions = "drive/My Drive/yunkai/solutions.csv"
  
  file = folder + '/' + fileName

  #get minmax but not really necessary
  df = pd.read_csv(pathToSolutions, names=['filepath','score','result','minmax','player','created_at'], header=0, sep="','", dtype={"score" :np.float64})
  
  df["filepath"] = df["filepath"].str[1:]
  df["created_at"] = df["created_at"].str[:-1]
  df["result"] = df["result"].apply(json.loads)
  df["minmax"] = df["minmax"].apply(json.loads)

  mins = df.loc[df['filepath'] == file]['minmax'].iloc[0][:2]
  maxs = df.loc[df['filepath'] == file]['minmax'].iloc[0][2:]

  allData = np.load(pathToData)
  # x = mins[0] + (allData[:,0] * (maxs[0] - mins[0]))
  # y = mins[1] + (allData[:,1] * (maxs[1] - mins[1]))
  x = allData[:,0]
  y = allData[:,1]

  #get cluster points
  myColors = dict()
  df = pd.read_csv(pathToCluster)
  color = 0  
  legend = [''] * len(df.columns)
  # print(legend)
  #mark color for each column
  for column in df.columns:
    myColors[column] = color
    legend[color] = column + ':' + str(color)
    color += 1

  #get the colormap
  colorMap = [None] * len(x)
  for index, row in df.iterrows():
    for i in range(len(row)):
      if(row[i] == True):
        colorMap[index] = i
        break
  
  # print("my colors")
  # print(colorMap)
  
  plt.scatter(x, y, c = colorMap, alpha=0.2, label=legend)
  plt.colorbar()
  plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)

  # plt.show()

  fileNameWithoutCSV = re.sub('\.csv', '', fileName)
  print(fileNameWithoutCSV)
  plt.savefig(outputPath + '/' + folder + '/' + fileNameWithoutCSV + '.png')
  

## Script to plot graph for all datasets (optional)



In [ ]:
def plotAllGraphs():
  clrsPath = 'drive/My Drive/yunkai/clrs'
  outputPath = 'drive/My Drive/yunkai/output'
  folders = listdir(clrsPath)
  for folder in folders:
    if not os.path.exists(outputPath + '/' + folder):
      os.makedirs(outputPath + '/' + folder)
    files = listdir(clrsPath + '/' + folder)
    for fileName in files:
      full_file_path = folder + '/' + fileName
      print(full_file_path)
      plotGraph(folder, fileName, outputPath)

## Get all games for the same dataset and add them in a shapefile

In [ ]:
def get_all_games_of_dataset(full_file_path):
  print(full_file_path)
  full_file_path_csv = full_file_path + '.csv'

  df = pd.read_csv(scaled_solutions_path)
  df["filepath"] = df["filepath"].str[1:]
  df["created_at"] = df["created_at"].str[:-1]
  df["result"] = df["result"].apply(json.loads)
  df["minmax"] = df["minmax"].apply(json.loads)

  # get all games of this dataset from all players
  output = []
  for index, row in df.iterrows():
    if row['filepath'] == full_file_path_csv:
      number_of_games += 1
      number_of_polygons += len(row['result']['polygons'])
      for polygon in row['result']['polygons']:
        coord = polygon['vertices']
        coord = np.array(coord)
        # times 10000 to scale up for later
        coord = [i*10000 for i in coord]
        # transform it into a shapefile format
        coord=[tuple(i) for i in coord]
        output.append({
            'type': 'Polygon',
            'coordinates': [coord]
        })
  print("numer of polygons is of length" + str(number_of_polygons))
  if len(output) == 0:
    return (output, 0)
  average = round(number_of_polygons/number_of_games)
  # in case if the average is 1, assume that it is impossible and round it up to 2
  if average == 1:
    average += 1
  print("average number of polygons is: " + str(number_of_polygons/number_of_games) + " rounded: " + str(average))
  # return all the polygons and the average number of polygons found by each player
  return (output, average)

## Rasterization method to generate consensus
## funtion that generates all the consensus polygons for all datasets, does not work as well compared to simple matplotlib

In [ ]:
def get_all_data():
  output_path = "drive/My Drive/yunkai/output"
  path_to_consensus_json = "drive/My Drive/yunkai/consensus.json"
  folders = listdir(clrs_path)
  for folder in folders:
    # create consensus folder for each data group
    if not os.path.exists(consensus_path + '/' + folder):
      os.makedirs(consensus_path + '/' + folder)
    all_files = listdir(clrs_path + '/' + folder)
    for file_name in all_files:
      file_name_without_csv = re.sub('\.csv', '', file_name)
      full_file_path = folder + '/' + file_name_without_csv
      # create consensus folder for each file_name too
      if not os.path.exists(consensus_path + '/' + folder + '/' + file_name_without_csv):
        os.makedirs(consensus_path + '/' + folder + '/' + file_name_without_csv)
      # this function fetches all games played on a specified dataset
      (shapes,average) = get_all_games_of_dataset(full_file_path)
      # if there isn't any shape returned from the dataset, then the game has not been played yet
      if len(shapes) == 0 :
        print("this game hasn't been played yet")
        continue  
      max_shape = cascaded_union([shape(s) for s in shapes])
      minx, miny, maxx, maxy = max_shape.bounds
      # change it to 0.001 for precision
      dx = dy = 1.0  # grid resolution
      lenx = dx * (ceil(maxx / dx) - floor(minx / dx))
      leny = dy * (ceil(maxy / dy) - floor(miny / dy))
      assert lenx % dx == 0.0
      assert leny % dy == 0.0
      nx = int(lenx / dx)
      ny = int(leny / dy)
      gt = Affine(dx, 0.0, dx * floor(minx / dx),0.0, -dy, dy * ceil(maxy / dy))
      pa = np.zeros((ny, nx), 'd')
      count = 0
      for s in shapes:
        try:
          r = features.rasterize([s], (ny, nx), transform=gt)
          pa[r > 0] += 1
          count += 1
        except:
          pass
      pa /= count  # normalise values
      plt.imshow(pa)
      plt.show()
      spa, sgt = gaussian_blur(pa, gt, 100)
      number_good_size_poly = 0
      decrement = 0
      best_threshold = 0
      best_average_size = 0
      while decrement <= 0.4: 
        plt.clf()
        thresh = 0.5-decrement  # median
        decrement += 0.05
        print("threshold is now: " + str(thresh))
        pm = np.zeros(spa.shape, 'B')
        pm[spa > thresh] = 1
        poly_shapes = []
        for sh, val in features.shapes(pm, transform=sgt):
          if val == 1:
              poly_shapes.append(shape(sh))
        if not any(poly_shapes):
          print("not good threshold to find shapes, decrement it")
          continue
        avg_poly = cascaded_union(poly_shapes)
        # Simplify the polygon
        simp_poly = avg_poly.simplify(sqrt(dx**2 + dy**2))
        simp_shape = mapping(simp_poly)
        number_good_size_poly = 0
        print("length of list of simp_shapes" + str(len(list(simp_shape['coordinates']))))
        # it's just one big cluster, not good
        if (len(list(simp_shape['coordinates']))) == 1 :
          print("length of 1 doesn't work, it is just a big cluster")
          continue
        list_of_areas = []
        total_size = 0
        to_list_simp_shape = [list(i) for i in list(simp_shape['coordinates'])]
        for list_of_tuples in to_list_simp_shape:
          if remove_layer_of_parenthesis(list_of_tuples):
            list_of_tuples = list(list(list_of_tuples)[0])
          list_of_lists = [list(i) for i in list_of_tuples]
          xs, ys = zip(*list_of_lists)
          xs = np.array(xs)
          ys = np.array(ys)
          if PolyArea(xs,ys) > 1000000:
            # print(list_of_lists)
            list_of_areas.append(PolyArea(xs,ys))
            total_size += PolyArea(xs,ys)
            plt.plot(xs, ys, c=np.random.rand(3, ), alpha=0.5)
            number_good_size_poly += 1
        print("number of polygons of good size: " + str(number_good_size_poly))
        if number_good_size_poly == average:
          average_size = 1
          for area in list_of_areas:
            average_size *= area/total_size
          if average_size > best_average_size:
            best_average_size = average_size
            best_threshold = thresh
        else :
          print("this file can't find the good size")
      if best_threshold != 0:
        plt.clf()
        thresh = best_threshold
        pm = np.zeros(spa.shape, 'B')
        pm[spa > thresh] = 1
        poly_shapes = []
        for sh, val in features.shapes(pm, transform=sgt):
          if val == 1:
              poly_shapes.append(shape(sh))
        avg_poly = cascaded_union(poly_shapes)
        # Simplify the polygon
        simp_poly = avg_poly.simplify(sqrt(dx**2 + dy**2))
        simp_shape = mapping(simp_poly)
        print("length of list of simp_shapes" + str(len(list(simp_shape['coordinates']))))
        # it's just one big cluster, not good
        if (len(list(simp_shape['coordinates']))) == 1 :
          print("length of 1 doesn't work")
          continue
        list_of_areas = []
        listToSave = []
        to_list_simp_shape = [list(i) for i in list(simp_shape['coordinates'])]
        count_poly = 0
        for list_of_tuples in to_list_simp_shape:
          if remove_layer_of_parenthesis(list_of_tuples):
            list_of_tuples = list(list(list_of_tuples)[0])
          list_of_lists = [list(i) for i in list_of_tuples]
          xs, ys = zip(*list_of_lists)
          xs = np.array(xs)
          ys = np.array(ys)
          if PolyArea(xs,ys) > 1000000:
            # print(list_of_lists)
            list_of_areas.append(PolyArea(xs,ys))
            plt.plot(xs, ys, c=np.random.rand(3, ), alpha=0.5)
            listToSave.append([])
            listToSave[count_poly] = (list_of_lists)
            print("count is " + str(count_poly))
            print(listToSave)
            count_poly += 1
        print("length of list to save")
        print(len(listToSave))
        # df.append({
        #     "filepath": folder + '/' + file_name_without_csv,
        #     "consensus polygons": listToSave
        # }, ignore_index=True)
        print(listToSave)
        with open(path_to_consensus_json, "r+") as json_file:
          data = json.load(json_file)
          data.append({
            "filepath": folder + '/' + file_name_without_csv,
            "consensus polygons": listToSave
          })
          print(data)
          with open(path_to_consensus_json, 'w') as f: 
            json.dump(data, f) 
        print("got it")
        name = file_name_without_csv + ".png"
        plt.savefig(output_path + '/' + folder + '/' + name)
        plt.clf()
        
      print("done for all files in this folder :)")

In [ ]:
def get_all_games_of_dataset(full_file_path):
  print(full_file_path)
  full_file_path_csv = full_file_path + '.csv'
  path = "drive/My Drive/yunkai/solutions.csv"

  df = pd.read_csv(path, names=['filepath','score','result','minmax','player','created_at'], header=0, sep="','", dtype={"score" :np.float64})
  df["filepath"] = df["filepath"].str[1:]
  df["created_at"] = df["created_at"].str[:-1]
  df["result"] = df["result"].apply(json.loads)
  df["minmax"] = df["minmax"].apply(json.loads)

  # get all games of this dataset from all players
  output = []
  number_of_games = 0;
  number_of_polygons = 0;
  for index, row in df.iterrows():
    if row['filepath'] == full_file_path_csv:
      number_of_games += 1
      number_of_polygons += len(row['result']['polygons'])
      for polygon in row['result']['polygons']:
        coord = polygon['vertices']
        coord = np.array(coord)
        # times 10000 to scale up for later
        coord = [i*10000 for i in coord]
        # transform it into a shapefile format
        coord=[tuple(i) for i in coord]
        output.append({
            'type': 'Polygon',
            'coordinates': [coord]
        })
  print("numer of polygons is of length" + str(number_of_polygons))
  if len(output) == 0:
    return (output, 0)
  average = round(number_of_polygons/number_of_games)
  # in case if the average is 1, assume that it is impossible and round it up to 2
  if average == 1:
    average += 1
  print("average number of polygons is: " + str(number_of_polygons/number_of_games) + " rounded: " + str(average))
  # return all the polygons and the average number of polygons found by each player
  return (output, average)

## Scale array to 1

In [ ]:
def scale_down (folder, file_name) :
  path_to_data = "drive/My Drive/yunkai/data/" + folder + '/' + file_name + ".csv.npy"
  allData = np.load(path_to_data)
  print(allData)
  xs, ys = zip(*allData)
  minX = min(xs)
  minY = min(ys)
  maxX = max(xs)
  maxY = max(ys)
  # to scale between 0 and 1
  xs = (xs - minX) / (maxX - minX)
  ys = (ys - minY) / (maxY - minY)
  # xs = np.asarray(xs)
  # ys = np.asarray(ys)
  # print(xs)
  # print(ys)
  scaled = np.vstack((xs, ys))
  scaled = np.transpose(scaled)
  print(folder + "/" + file_name)
  print(scaled)
  saved_path = "drive/My Drive/yunkai/data/" + folder + '/scaled_' + file_name + ".csv.npy"
  np.save(saved_path, scaled)

In [ ]:
def scale_all_data():
  path_to_data = "drive/My Drive/yunkai/data/"

  folders = listdir(path_to_data)
  for folder in folders:
    all_files = listdir(path_to_data + '/' + folder)
    for file_name in all_files:
      file_name = re.sub('\.npy', '', file_name)
      file_name = re.sub('\.csv', '', file_name)
      scale_down(folder, file_name)


## Gaussian Blur

In [ ]:
from scipy.signal import fftconvolve

def gaussian_blur(in_array, gt, size):
    """Gaussian blur, returns tuple `(ar, gt2)` that have been expanded by `size`"""
    # expand in_array to fit edge of kernel; constant value is zero
    padded_array = np.pad(in_array, size, 'constant')
    # build kernel
    x, y = np.mgrid[-size:size + 1, -size:size + 1]
    g = np.exp(-(x**2 / float(size) + y**2 / float(size)))
    g = (g / g.sum()).astype(in_array.dtype)
    # do the Gaussian blur
    ar = fftconvolve(padded_array, g, mode='full')
    # convolved increased size of array ('full' option); update geotransform
    gt2 = Affine(
        gt.a, gt.b, gt.xoff - (2 * size * gt.a),
        gt.d, gt.e, gt.yoff - (2 * size * gt.e))
    return ar, gt2

## Preprocess data and generate k clusters based on mean shift s.t. it represents 10 games "played by gamers"


In [ ]:
def generate_all_ml_polygons():
  folders = listdir(data_path)
  for folder in folders:
    all_files = listdir(data_path + '/' + folder)
    for file_name in all_files:
      print(folder + '/' + file_name)
      X = np.load(data_path + '/' + folder + '/' + file_name)
      range_n_clusters = [2, 3, 4, 5]
      max_average = 0
      best_cluster_number = -1
      # for run_number in range(3):
      for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
        plt.cla()
        plt.close()
        plt.clf()
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_size_inches(18, 7)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-0.1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        clusterer = KMeans(n_clusters=n_clusters)
        cluster_labels = clusterer.fit_predict(X)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              "The average silhouette_score is :", silhouette_avg)
        if silhouette_avg > 0.65 and max_average < silhouette_avg:
          max_average = silhouette_avg
          best_cluster_number = n_clusters
      if best_cluster_number != -1:
        n_clusters = best_cluster_number
        plt.cla()
        plt.close()
        plt.clf()
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_size_inches(18, 7)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-0.1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        clusterer = KMeans(n_clusters=n_clusters)
        cluster_labels = clusterer.fit_predict(X)
        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              "The average silhouette_score is :", silhouette_avg)
        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X, cluster_labels)

        y_lower = 10
        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters.")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # 2nd Plot showing the actual clusters formed
        colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
        ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                    c=colors, edgecolor='k')
        plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                      "with n_clusters = %d" % n_clusters),
                    fontsize=14, fontweight='bold')
        plt.show()
        # # check variance using gaussian kde for each polygon
        clusters = dict()
        for i in range(len(colors)):
          checksum = np.sum(colors[i])
          if not checksum in clusters:
            clusters[checksum] = []
          clusters[checksum].append(X[i].tolist())
        exterior_bounds = []
        for cluster in clusters:
          plt.cla()
          plt.clf()
          coordinates = np.array(clusters[cluster])
          x = coordinates[:, 0]
          y = coordinates[:, 1]
          # Define the borders
          values = np.vstack([x, y])
          kernel = st.gaussian_kde(values)
          # Define the borders
          deltaX = (max(x) - min(x))/10
          deltaY = (max(y) - min(y))/10
          xmin = min(x) - deltaX
          xmax = max(x) + deltaX
          ymin = min(y) - deltaY
          ymax = max(y) + deltaY
          # Create meshgrid
          xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j] 

          positions = np.vstack([xx.ravel(), yy.ravel()])
          values = np.vstack([x, y])
          f = np.reshape(kernel(positions).T, xx.shape)
          fig = plt.figure(figsize=(8,8))
          ax = fig.gca()
          ax.set_xlim(xmin, xmax)
          ax.set_ylim(ymin, ymax)
          cfset = ax.contourf(xx, yy, f, cmap='coolwarm')
          ax.imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
          cset = ax.contour(xx, yy, f, colors='k')
          ax.clabel(cset, inline=1, fontsize=10)
          ax.set_xlabel('X')
          ax.set_ylabel('Y')
          plt.title('2D Gaussian Kernel density estimation')
          plt.figure(figsize=(8,8))
          for j in range(len(cset.allsegs)):
            for ii, seg in enumerate(cset.allsegs[j]):
              if j == 1 and ii == 0:
                exterior_bounds.append(seg.tolist())
              plt.plot(seg[:,0], seg[:,1], '.-', label=f'Cluster{j}, level{ii}')
          plt.legend()
          plt.show()
          # plt.figure(figsize=(8,8))
          # for j in range(len(cset.allsegs)):
          #   for ii, seg in enumerate(cset.allsegs[j]):
              
        np.save(consensus_path + '/' + folder + '-' + file_name + '_machine_generated', exterior_bounds)


## Test using meanshift

In [ ]:
def mean_shift_implementation():
  x_shrink = 0.4
  y_shrink = 0.4
  # The generation will use all the available datasets that we have.
  # It will traverse all the folders and all the files present in the clrs folder
  # in order to obtain all the datasets.
  folders = listdir(clrs_path)
  for folder in folders:
    all_files = listdir(clrs_path + '/' + folder)
    for file_name in all_files:
      file_name_without_csv = re.sub('\.csv', '', file_name)
      full_file_path = folder + '/' + file_name_without_csv

      # This function fetches all games played on a specified dataset
      polygons = random_plot_get_all_games(full_file_path)

      # obtain the max and min coordinate to scale the value down to a 1x1 square
      minX = 2147483647
      minY = 2147483647
      maxX = -2147483647
      maxY = -2147483647
      for polygon in polygons:
        for coordinates in polygon:
          minX = min(minX, coordinates[0])
          maxX = max(maxX, coordinates[0])
          minY = min(minY, coordinates[1])
          maxY = max(maxY, coordinates[1])

      # scale the coordinates between the range of [0,1]
      scaled_polygons = []
      for polygon in polygons:
        temp_list = []
        for coordinates in polygon:
          x_val = (coordinates[0] - minX)/ (maxX - minX)
          y_val = (coordinates[1] - minY)/ (maxY - minY)
          temp_list.append(tuple([x_val, y_val]))
        scaled_polygons.append(temp_list)

      # If there is less than 30 games played, do not generate the consensus polygon as it is too small sample size
      if len(polygons) < 30 :
        continue

      polygons = scaled_polygons
      # totalArea = getTotalArea(polygons)
      # print("This game has a total area of" + str(totalArea))

      # Get random points per polygon in order to later plot density plot
      scatter_plot = []

      # shrink down size of each polygon to get a better separation
      for polygon in polygons:
        xs, ys = zip(*polygon)
        xs = np.array(xs)
        ys = np.array(ys)
        x_center = 0.5 * min(xs)  + 0.5 * max(xs)
        y_center = 0.5 * min(ys)  + 0.5 * max(ys)
        new_xs = [(i - x_center) * (1 - x_shrink) + x_center for i in xs]
        new_ys = [(i - y_center) * (1 - y_shrink) + y_center for i in ys]
        # create list of new coordinates
        polygon = zip(new_xs, new_ys)
        poly = Polygon(polygon)
        polygonArea = PolyArea(xs,ys)
        for i in range(int(100 * (polygonArea))):
          point_in_poly = random_point_in_polygon(poly)
          scatter_plot.append([point_in_poly.x, point_in_poly.y])
        
      # Store it in a numpy array so that it is easier to fetch all x coordinates and all y coordinates later
      X = np.array(scatter_plot)
      bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=500)

      ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
      ms.fit(X)
      labels = ms.labels_
      cluster_centers = ms.cluster_centers_

      labels_unique = np.unique(labels)
      n_clusters_ = len(labels_unique)

      print("number of estimated clusters : %d" % n_clusters_)

      # #############################################################################
      # Plot result

      plt.figure(1)
      plt.clf()

      colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
      for k, col in zip(range(n_clusters_), colors):
          my_members = labels == k
          cluster_center = cluster_centers[k]
          plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
          plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
                  markeredgecolor='k', markersize=14)
      plt.title('Estimated number of clusters: %d' % n_clusters_)
      plt.show()
          

## DBSCAN Implementation

In [ ]:
def DBSCAN_implementation():
  x_shrink = 0.4
  y_shrink = 0.4
  # The generation will use all the available datasets that we have.
  # It will traverse all the folders and all the files present in the clrs folder
  # in order to obtain all the datasets.
  folders = listdir(clrs_path)
  for folder in folders:
    all_files = listdir(clrs_path + '/' + folder)
    for file_name in all_files:
      file_name_without_csv = re.sub('\.csv', '', file_name)
      full_file_path = folder + '/' + file_name_without_csv

      # This function fetches all games played on a specified dataset
      polygons = random_plot_get_all_games(full_file_path)

      # obtain the max and min coordinate to scale the value down to a 1x1 square
      minX = 2147483647
      minY = 2147483647
      maxX = -2147483647
      maxY = -2147483647
      for polygon in polygons:
        for coordinates in polygon:
          minX = min(minX, coordinates[0])
          maxX = max(maxX, coordinates[0])
          minY = min(minY, coordinates[1])
          maxY = max(maxY, coordinates[1])

      # scale the coordinates between the range of [0,1]
      scaled_polygons = []
      for polygon in polygons:
        temp_list = []
        for coordinates in polygon:
          x_val = (coordinates[0] - minX)/ (maxX - minX)
          y_val = (coordinates[1] - minY)/ (maxY - minY)
          temp_list.append(tuple([x_val, y_val]))
        scaled_polygons.append(temp_list)

      # If there is less than 30 games played, do not generate the consensus polygon as it is too small sample size
      if len(polygons) < 30 :
        continue

      polygons = scaled_polygons
      # totalArea = getTotalArea(polygons)
      # print("This game has a total area of" + str(totalArea))

      # Get random points per polygon in order to later plot density plot
      scatter_plot = []

      # shrink down size of each polygon to get a better separation
      for polygon in polygons:
        xs, ys = zip(*polygon)
        xs = np.array(xs)
        ys = np.array(ys)
        x_center = 0.5 * min(xs)  + 0.5 * max(xs)
        y_center = 0.5 * min(ys)  + 0.5 * max(ys)
        new_xs = [(i - x_center) * (1 - x_shrink) + x_center for i in xs]
        new_ys = [(i - y_center) * (1 - y_shrink) + y_center for i in ys]
        # create list of new coordinates
        polygon = zip(new_xs, new_ys)
        poly = Polygon(polygon)
        polygonArea = PolyArea(xs,ys)
        for i in range(int(100 * (polygonArea))):
          point_in_poly = random_point_in_polygon(poly)
          scatter_plot.append([point_in_poly.x, point_in_poly.y])
        
      # Store it in a numpy array so that it is easier to fetch all x coordinates and all y coordinates later
      X = np.array(scatter_plot)
      # Compute DBSCAN
      db = DBSCAN(eps=0.2, min_samples=200).fit(X)
      core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
      core_samples_mask[db.core_sample_indices_] = True
      labels = db.labels_

      # Number of clusters in labels, ignoring noise if present.
      n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
      n_noise_ = list(labels).count(-1)

      # #############################################################################
      # Plot result
      import matplotlib.pyplot as plt

      # Black removed and is used for noise instead.
      unique_labels = set(labels)
      colors = [plt.cm.Spectral(each)
                for each in np.linspace(0, 1, len(unique_labels))]
      for k, col in zip(unique_labels, colors):
          if k == -1:
              # Black used for noise.
              col = [0, 0, 0, 1]

          class_member_mask = (labels == k)

          xy = X[class_member_mask & core_samples_mask]
          plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                  markeredgecolor='k', markersize=14)

          xy = X[class_member_mask & ~core_samples_mask]
          plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                  markeredgecolor='k', markersize=6)

      plt.title('Estimated number of clusters: %d' % n_clusters_)
      plt.show()

In [ ]:
# DBSCAN_implementation()